In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from os.path import abspath
import re
import os
import matplotlib.image
from functools import partial
from IPython.display import Image
from sklearn import linear_model
from statistics import median 
# import plotly.offline as py
import seaborn as sns
# import plotly.figure_factory as ff
import pingouin as pg
import itertools
from tqdm import tqdm 
import time 

In [ ]:
# 'trippub.csv' can be downloaded from 
# Federal Highway Administration. (2017). 2017 National Household Travel Survey, U.S. Department of Transportation, Washington, DC. 
# Available online: https://nhts.ornl.gov.
trip17 = pd.read_csv('trippub.csv')
trip17.head()

In [ ]:
# Drop all zero values of trip dataset
trip17 = trip17.loc[trip17.WTTRDFIN > 0]

# Remove outliers which are not within 3 standard deviations from mean of trip dataset
trip17 = trip17[
    np.abs(trip17.WTTRDFIN - trip17.WTTRDFIN.mean()) <= (3*trip17.WTTRDFIN.std())
]

In [ ]:
trip17_1pv = trip17.groupby(['HOUSEID','VEHID'], as_index=False)['VMT_MILE'].sum()

In [ ]:
trip17_process=trip17_1pv.groupby(['HOUSEID']).agg(list)

In [ ]:
trip17_process['freq_car_index']=trip17_process.agg(lambda x: x['VEHID'][np.argmax(x['VMT_MILE'])],axis=1)

In [ ]:
trip17_process=trip17_process.reset_index()

In [ ]:
trip17_process.head()

In [ ]:
trip17.head()

In [ ]:
trip17_final=trip17.merge(trip17_process[['HOUSEID','freq_car_index']],on=['HOUSEID'],how='left')

In [ ]:
trip17_final['selected_car']=trip17_final['freq_car_index']==trip17_final['VEHID']

In [ ]:
trip17_final[['freq_car_index','VEHID','selected_car']].head()

In [ ]:
trip17_final = trip17_final[trip17_final['selected_car'] == True]

In [ ]:
trip17_final.head()

In [ ]:
simu = pd.read_csv('outputs/new_simulation_result_income_162500.csv')

In [ ]:
simu.head()

In [ ]:
trip17_pred=trip17_final.merge(simu,on=['HOUSEID'],how='left')

In [ ]:
trip17_pred.head()

In [ ]:
### Trip distance in miles for personally driven vehicle trips
### new Trip Distance Distribution with new categrarical CENSUS DIVISIONS using Dummy Variables
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

### VMT Analysis

In [ ]:
trip17_vmt = trip17_pred[trip17_pred['VMT_MILE']>0]
trip17_vmt = trip17_vmt[trip17_vmt['TRVLCMIN']>0]

In [ ]:
trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]

for i in range(24):
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)


In [ ]:
trip17_vmt_T.head()

In [ ]:
avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
avg_vmt_F = [[0 for i in range(24)] for j in range(100)]
avg_vmt = [[0 for i in range(24)] for j in range(100)]

# trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
# trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]
trip17_vmt_T = trip17_vmt_T[trip17_vmt_T['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt_F[trip17_vmt_F['MUDs_Predict_OW'] == 0]


#trip17_MUD_C['HHIncomes'].mask(trip17_MUD_C['HHFAMINC'] == i+1, income[i], inplace=True)
for i in range(24):
    #avg_vmt_T[0][i] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt_T[0][i] = trip17_vmt_T['VMT_Hr'+str(i+1)].mean()
    #avg_vmt_F[0][i] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt_F[0][i] = trip17_vmt_F['VMT_Hr'+str(i+1)].mean()
    avg_vmt[0][i] = trip17_vmt[(trip17_vmt['STRTTIME'] >= 100*i) & (trip17_vmt['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    

In [ ]:
for i in range(1,100):
#     trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW_'+str(i)] == 1]
#     trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW_'+str(i)] == 0]
    trip17_vmt_T = trip17_vmt_T[trip17_vmt_T['MUDs_Predict_OW_'+str(i)] == 1]
    trip17_vmt_F = trip17_vmt_F[trip17_vmt_F['MUDs_Predict_OW_'+str(i)] == 0]
    for j in range(24):
#         avg_vmt_T[i][j] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*j) & (trip17_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt_T[i][j] = trip17_vmt_T['VMT_Hr'+str(j+1)].mean()
#         avg_vmt_F[i][j] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*j) & (trip17_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt_F[i][j] = trip17_vmt_F['VMT_Hr'+str(j+1)].mean()
        avg_vmt[i][j] = trip17_vmt[(trip17_vmt['STRTTIME'] >= 100*j) & (trip17_vmt['STRTTIME']< 100*(j+1))].VMT_MILE.mean()

# for i in range(24):
#     trip17_vmt_T.loc[trip17_vmt_T['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120
#     trip17_vmt_F.loc[trip17_vmt_F['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120

In [ ]:
col_list = range(24)
df_avg_vmt_T = pd.DataFrame(data=avg_vmt_T, columns=col_list)
df_avg_vmt_F = pd.DataFrame(data=avg_vmt_F, columns=col_list)
df_avg_vmt = pd.DataFrame(data=avg_vmt, columns=col_list)
df_avg_vmt_T

consider all trips

In [ ]:
df_avg_vmt_T.fillna(0, inplace=True)
df_avg_vmt_F.fillna(0, inplace=True)
df_avg_vmt.fillna(0,inplace=True)

In [ ]:
# function for setting the colors of the box plots pairs
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')
  

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Time of Day Average VMT for all personally driven vehicle trips (Miles)', fontsize=28)
 
# data required for plotting
all_data = [df_avg_vmt_T[i] for i in range(24)]
all_data_F = [df_avg_vmt_F[i] for i in range(24)]

# plot box plot
bp2 = plt.boxplot(all_data,positions=np.array(range(1,len(all_data)+1))*2-0.3)
set_box_color(bp2, '#0000FF')


# plot box plot
bp4 = plt.boxplot(all_data_F,positions=np.array(range(1,len(all_data)+1))*2+0.3)
#axs[1][1].set_title('Box plot')
set_box_color(bp4, '#FF0000')


plt.plot([], c='#0000FF', label="MUDs' Residents")
plt.plot([], c='#FF0000', label="Others' Residents")
plt.legend()
# adding horizontal grid lines

plt.xticks(range(1, len(tod)*2+1, 2), tod)
plt.xticks(rotation=45)        
plt.xlabel('Time of Day',fontdict=font)
plt.ylabel('VMT(Miles)',fontdict=font)

plt.show()

consider trips with positive vmts excludingly

In [ ]:
trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]
#trip17_vmt = trip17_vmt[trip17_vmt['TRVLCMIN']>0]

In [ ]:
trip17_vmt_T.head()

In [ ]:
for i in range(24):
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
                                               

In [ ]:
for i in range(24):
    trip17_vmt_T.loc[trip17_vmt_T['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120
    trip17_vmt_F.loc[trip17_vmt_F['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120

In [ ]:
trip17_vmt_T[['HOUSEID','VMT_Hr16','VMT_MILE']].head(10)

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')
    

In [ ]:
trip17_vmt_T[trip17_vmt_T['VMT_Hr'+str(1+1)] != 0]['VMT_Hr2'].head()

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Time of Day VMT for personally driven vehicle trips (vmt>0)', fontsize=28)
 
# data required for plotting
all_data = [trip17_vmt_T[trip17_vmt_T['VMT_Hr'+str(i+1)] != 0]['VMT_Hr'+str(i+1)] 
            for i in range(24)]
all_data_F = [trip17_vmt_F[trip17_vmt_F['VMT_Hr'+str(i+1)]!=0]['VMT_Hr'+str(i+1)] 
            for i in range(24)]

# plot box plot
flierprops1 = dict(marker='o', markerfacecolor='blue', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp2 = plt.boxplot(all_data,positions=np.array(range(1,len(all_data)+1))*2-0.3, flierprops=flierprops1)
set_box_color(bp2, '#0000FF')


# plot box plot
flierprops2 = dict(marker='o', markerfacecolor='r', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp4 = plt.boxplot(all_data_F,positions=np.array(range(1,len(all_data)+1))*2+0.3, flierprops=flierprops2)
#axs[1][1].set_title('Box plot')
set_box_color(bp4, '#FF0000')


plt.plot([], c='#0000FF', label="MUDs' Residents")
plt.plot([], c='#FF0000', label="Others' Residents")
plt.legend(loc='upper left', bbox_to_anchor=(0.9, 1.06))
# adding horizontal grid lines

plt.xticks(range(1, len(tod)*2+1, 2), tod)
plt.xticks(rotation=45)        
plt.xlabel('Time of Day (Hrs)',fontdict=font)
plt.ylabel('VMT (Miles)',fontdict=font)

plt.show();

In [ ]:
list_avg_vmt_T = []
list_avg_vmt_F = []
for i in range(24):
    list_avg_vmt_T.append(all_data[i].mean())
    list_avg_vmt_F.append(all_data_F[i].mean())

plot figure 2

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')

plt.figure(figsize =(16,8),dpi=600)

plt.plot(tod, list_avg_vmt_T, label='MUDs');
plt.plot(tod, list_avg_vmt_F, label='Others');
#plt.plot(tod, avg_vmt[0], label='Alls');

plt.xlabel('Time of day (hrs)', fontsize=20)
plt.ylabel('Average VMT (miles)', fontsize=20)
plt.xticks(fontsize=18,rotation=45)
plt.yticks(fontsize=18)
plt.title('Time-of-day average VMT per household trip', fontsize=28)
plt.legend(loc='upper right',fontsize=24)


plt.show()

### Distribution of dwell time

In [ ]:
### Drop arrows where "VEHID" == "-9"&"-1", which means "Not ascertained" and "Appropriate skip" respectively
trip17_pred_veh_determined = trip17_pred[trip17_pred['VEHID'] != -1]
trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined['VEHID'] != -9]

In [ ]:
trip17_pred_veh_determined[['HOUSEID','VEHID']].head()

In [ ]:
type(trip17_pred_veh_determined['WHYTRP1S'])

In [ ]:
### classification of "WHYTRP1S", which is "Trip purpose summary"
trip17_pred_veh_determined['WHYTRP1S_GENERAL'] = np.where(trip17_pred_veh_determined['WHYTRP1S']== 1, 'Home', 
                                             np.where(trip17_pred_veh_determined['WHYTRP1S']== 10, 'Work','Public'))


In [ ]:
### analyze time before the starttime of the first trip, use 
### "WHYFROM", which is "Trip Origin Purpose"
trip17_pred_veh_determined['WHYFROM_GENERAL'] = np.where((trip17_pred_veh_determined['WHYFROM']== 1) | (trip17_pred_veh_determined['WHYFROM']== 2), 'Home', 
                                             np.where((trip17_pred_veh_determined['WHYFROM']== 3) | (trip17_pred_veh_determined['WHYFROM']== 4), 'Work','Public'))


In [ ]:
trip17_pred_veh_determined[['HOUSEID','VEHID','WHYTRP1S_GENERAL','WHYFROM_GENERAL']].head()

In [ ]:
trip17_pred_veh_determined.drop(columns='PERSONID',inplace=True)

In [ ]:
trip17_pred_veh_determined.head()

In [ ]:
trip17_pred_veh_determined.drop_duplicates(subset=['HOUSEID','TDTRPNUM','STRTTIME','ENDTIME',
                                                  ],inplace = True)

In [ ]:
trip17_pred_veh_determined.head(20)

In [ ]:
# trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined.DWELTIME != -9]

In [ ]:
### pick out the columns needed for location classification during time of day 
### from the whole "trip17_pred_veh_determined" dataframe
trip17_pred_veh_determined_part = trip17_pred_veh_determined[['HOUSEID','VEHID','STRTTIME','ENDTIME','TRVLCMIN','DWELTIME','WHYFROM_GENERAL','WHYTRP1S_GENERAL','MUDs_Predict_OW','CENSUS_D']]
trip17_pred_veh_determined_part = trip17_pred_veh_determined_part.groupby(['HOUSEID','VEHID'], as_index=False).agg(lambda x: list(x))

In [ ]:
type(trip17_pred_veh_determined_part)

In [ ]:
temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part.loc[4,'HOUSEID']]
temp.reset_index(inplace=True)
temp

In [ ]:

trip17_pred_veh_determined_part_1 = trip17_pred_veh_determined_part
trip17_pred_veh_determined_part_1.reset_index(inplace=True)

for k in range(len(trip17_pred_veh_determined_part_1)):
    temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part_1.loc[k,'HOUSEID']]
    temp.reset_index(inplace=True)
    for i in range(24):
        for count in range(len(temp)):
            ### trip time in a hour
            if ((temp.loc[count,'STRTTIME']>=i*100) & (temp.loc[count,'ENDTIME']<=(i+1)*100)):
                trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = 'Driving'
                #aks[i]='Driving'
            ### trip time over a hour
            if ((temp.loc[count,'STRTTIME']<=i*100) & (temp.loc[count,'ENDTIME']>=(i+0)*100)):
                trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = 'Driving'
                #aks[i]='Driving'
            if count+2<=len(temp):
                if ((i*100 >= temp.loc[count,'ENDTIME']) & (i*100 <= temp.loc[count+1,'STRTTIME'])):
                    trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[count,'WHYTRP1S_GENERAL']
                    #aks[i]=temp.loc[count,'WHYTRP1S_GENERAL']
        if (temp.loc[0,'STRTTIME']>i*100):
            trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
            #aks[i]=temp.loc[0,'WHYFROM_GENERAL']
        if (temp.tail(1)['ENDTIME'].to_list()[0] < i*100):
            trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[len(temp)-1,'WHYTRP1S_GENERAL']
            #aks[i]=temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0]
    
for i in tqdm (range (101),  
               desc="Loading…",  
               ascii=False, ncols=75): 
    time.sleep(0.01) 
print("Complete.") 
    



In [ ]:
# trip17_pred_veh_determined_part_1.to_csv('trip17_pred_veh_determined_part_1.csv')

plot figure 3

In [ ]:
# trip17_pred_veh_determined_part_1 = pd.read_csv('outputs/trip17_pred_veh_determined_part_1.csv')

In [ ]:
dwell_process = trip17_pred_veh_determined_part_1[['HOUSEID','VEHID','MUDs_Predict_OW',
                                                  'location_hr_1','location_hr_2','location_hr_3','location_hr_4',
                                                  'location_hr_5','location_hr_6','location_hr_7','location_hr_8',
                                                  'location_hr_9','location_hr_10','location_hr_11','location_hr_12',
                                                  'location_hr_13','location_hr_14','location_hr_15',
                                                  'location_hr_16','location_hr_17','location_hr_18',
                                                  'location_hr_19','location_hr_20','location_hr_21',
                                                  'location_hr_22','location_hr_23','location_hr_24']]

In [ ]:
dwell_process.head()

In [ ]:
dwell_process.iloc[0,:]=='Home'

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_HOME'] = sum(dwell_process.iloc[i,:]=='Home')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_WORK'] = sum(dwell_process.iloc[i,:]=='Work')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_PUBLIC'] = sum(dwell_process.iloc[i,:]=='Public')

In [ ]:
dwell_process.head(10)

In [ ]:
dwell_process['MUDs_Predict_OW'] = dwell_process['MUDs_Predict_OW'].astype(int)

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 1.0 in x)
dwell_process_mud = dwell_process[mask]
# dwell_process_mud = dwell_process[dwell_process['MUDs_Predict_OW']==1.0]

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 0.0 in x)
dwell_process_other = dwell_process[mask]
# dwell_process_other = dwell_process[dwell_process['MUDs_Predict_OW']==0.0]

In [ ]:
dwell_process_other.head()

In [ ]:
dwell_process_mud_home = dwell_process_mud['DWELL_HOME']
dwell_process_mud_work = dwell_process_mud['DWELL_WORK']
dwell_process_mud_public = dwell_process_mud['DWELL_PUBLIC']

dwell_process_other_home = dwell_process_other['DWELL_HOME']
dwell_process_other_work = dwell_process_other['DWELL_WORK']
dwell_process_other_public = dwell_process_other['DWELL_PUBLIC']

In [ ]:
dwell_process_mud_home.head()

In [ ]:
######################################################################################################
# REVISION:
######################################################################################################
fig = plt.figure(figsize = (16,8),dpi=600)
p1=sns.kdeplot(dwell_process_mud_home, shade=True, label = 'Home of MUDs',color = '#0000FF')
p1=sns.kdeplot(dwell_process_mud_work, shade=True,label = 'Workplace of MUDs',color = '#FF0000')
p1=sns.kdeplot(dwell_process_mud_public, shade=True,label = 'Public of MUDs',color ='#008000')
p1=sns.kdeplot(dwell_process_other_home, shade=True,label = 'Home of others',color = '#87CEFA',linestyle="--")
p1=sns.kdeplot(dwell_process_other_work, shade=True,label = 'Workplace of others',color ='#FFA07A',linestyle="--")
p1=sns.kdeplot(dwell_process_other_public, shade=True, label = 'Public of others',color = '#7CFC00',linestyle="--")
plt.title('Distribution of dwell time', fontsize=28)
plt.xlabel('Time of day (hrs)',fontsize=20);
plt.ylabel('Density',fontsize=20);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.legend(loc='best',fontsize=20)
plt.show();
# plt.savefig('FIGURE3.png',dpi=600)

plot figure 4

In [ ]:
# trip17_pred_veh_determined_part_1 = pd.read_csv('outputs/trip17_pred_veh_determined_part_1.csv')
trip17_pred_veh_determined_part_1.head()

In [ ]:
trip17_pred_veh_determined_part_1.drop(columns=['Unnamed: 0','index'],inplace=True)
trip17_pred_veh_determined_part_1.head()

In [ ]:
trip17_pred_veh_determined_part_mud = trip17_pred_veh_determined_part_1[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part_1['MUDs_Predict_OW']))[0])) == 1.0]
trip17_pred_veh_determined_part_other = trip17_pred_veh_determined_part_1[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part_1['MUDs_Predict_OW']))[0])) == 0.0]

In [ ]:
labels = []
for i in range(24):
    labels.append(str(i) + ':00 to ' + str(i+1) + ':00')
home_share_mud = []
work_share_mud = []
public_share_mud = []
driving_share_mud = []
home_share_other = []
work_share_other = []
public_share_other = []
driving_share_other = []
for i in range(24):
    home_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Home'])/len(trip17_pred_veh_determined_part_mud)
    home_share_mud.append(home_mud)
    work_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Work'])/len(trip17_pred_veh_determined_part_mud)
    work_share_mud.append(work_mud)
    public_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Public'])/len(trip17_pred_veh_determined_part_mud)
    public_share_mud.append(public_mud)
    driving_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Driving'])/len(trip17_pred_veh_determined_part_mud)
    driving_share_mud.append(driving_mud)
    
#     home_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
#                                                        == 'Home'])/len(trip17_pred_veh_determined_part_other)
#     home_share_other.append(home_other)
    work_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Work'])/len(trip17_pred_veh_determined_part_other)
    work_share_other.append(work_other)
    public_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Public'])/len(trip17_pred_veh_determined_part_other)
    public_share_other.append(public_other)
    driving_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Driving'])/len(trip17_pred_veh_determined_part_other)
    driving_share_other.append(driving_other)
    home_other = 1-work_other-public_other-driving_other
    home_share_other.append(home_other)
    

In [ ]:
home_share_mud_100 = []
for item in home_share_mud:
    home_share_mud_100.append(item*100)
home_share_mud = home_share_mud_100

home_share_other_100 = []
for item in home_share_other:
    home_share_other_100.append(item*100)
home_share_other = home_share_other_100

driving_share_mud_100 = []
for item in driving_share_mud:
    driving_share_mud_100.append(item*100)
driving_share_mud = driving_share_mud_100

driving_share_other_100 = []
for item in driving_share_other:
    driving_share_other_100.append(item*100)
driving_share_other = driving_share_other_100

work_share_mud_100 = []
for item in work_share_mud:
    work_share_mud_100.append(item*100)
work_share_mud = work_share_mud_100

work_share_other_100 = []
for item in work_share_other:
    work_share_other_100.append(item*100)
work_share_other = work_share_other_100

public_share_mud_100 = []
for item in public_share_mud:
    public_share_mud_100.append(item*100)
public_share_mud = public_share_mud_100

public_share_other_100 = []
for item in public_share_other:
    public_share_other_100.append(item*100)
public_share_other = public_share_other_100

In [ ]:
bar12 = np.add(home_share_mud, driving_share_mud).tolist()
bar123 = np.add(bar12, work_share_mud).tolist()

bar12other = np.add(home_share_other, driving_share_other).tolist()
bar123other = np.add(bar12other, work_share_other).tolist()
    
x = np.arange(len(labels))  # the label locations
width = 0.35       # the width of the bars: can also be len(x) sequence


fig = plt.figure(figsize=(16, 8),dpi=600)

plt.bar(x-width/2, home_share_mud, width, label='Home of MUDs',color='#00008B')
plt.bar(x-width/2, driving_share_mud, width, bottom=home_share_mud, label='Driving of MUDs',color='#0000FF')
plt.bar(x-width/2, work_share_mud, width, bottom=bar12, label='Work of MUDs',color='#00BFFF')
plt.bar(x-width/2, public_share_mud, width, bottom=bar123, label='Public of MUDs',color='#B0E0E6')

plt.bar(x+width/2, home_share_other, width, label='Home of others',color='#8B0000')
plt.bar(x+width/2, driving_share_other, width, bottom=home_share_other, label='Driving of others',color='#FF0000')
plt.bar(x+width/2, work_share_other, width, bottom=bar12other, label='Work of others',color='#F08080')
plt.bar(x+width/2, public_share_other, width, bottom=bar123other, label='Public of others',color='#FFA07A')


#df.plot.bar(ax=axes[1], stacked=True)

# plt.xticks(Rotation=45) 
plt.xlabel('Time of day (hrs)',fontsize=20)
plt.ylabel('Percentage of vehicles (%)',fontsize=20)
plt.xticks(fontsize=18, rotation=45)
plt.yticks(fontsize=18)
plt.title('Status and location of vehicles per time of day',fontsize=28)
plt.legend(loc='upper left', bbox_to_anchor=(0.95, 1.025), fontsize=16)

plt.show()
# fig.savefig('FIGURE_4.png',dpi=600)